In [120]:
import vk_api
import io
import json, codecs

In [128]:
# credentials: нужно указать логин/пароль к vk.com
# чтобы случайно не закомитить версию с логин/пароль - выносим эти данные в отдельный файл, который исключаем в .gitignore
credentialsFileName = "vk_login_password.txt" # формат файла: 1-я строка = логин, 2-я строка = пароль
# затираем значения, которые могли сохраниться с прошлых запусков (в notebook)
login = '+71231234567' 
password = ''

# загружаем значения из файла
try:
    loginFile = open(credentialsFileName, 'r')
    login = loginFile.readline().strip() # в формате: '+71231234567'
    password = loginFile.readline().strip()
except IOError:
    print ("No file with login/password: создайте файл " + credentialsFileName + ", в котором 1-я строка = логин, 2-я строка = пароль")

In [ ]:
# используем:
#  wrapper-lib vk_api: https://github.com/python273/vk_api
#  описание Api от vk: https://vk.com/dev/groups.search?params[q]=bitcoin&params[future]=0&params[market]=0&params[offset]=3&params[count]=3&params[v]=5.92

In [140]:
# параметры запуска
GROUP_KEYWORD = 'bitcoin' # интересуют сообщества, в названии которых есть такое слово
COUNT_GROUPS = 50 # обработаем столько сообществ
COUNT_MESSAGES_PER_GROUP = 100 # в каждом сообществе, со стены возьмем столько сообщений

In [50]:
# получаем соединение, api
vk_session = vk_api.VkApi(login, password)
vk_session.auth(token_only=True)
api = vk_session.get_api()

In [ ]:
# константы и правила, которые используем при работе с файлами
FOLDER_PREFIX = './files/' # folder, в которую складываем файлы
FILE_EXTENSION = '.txt' # расширение для json-файлов

def fileNameFunc(rawFileName):
    return FOLDER_PREFIX + rawFileName + FILE_EXTENSION

In [130]:
# Данные по сообществам - получаем по http
def obtainGroups(api, keyword, count):
    print("obtainGroups, keyword=" + keyword + ", count=" + str(count))
    results = api.groups.search(q=keyword, count=count)
    return results['items']

In [129]:
# получить N записей со стены сообщества/пользователя
def obtainWallItems(api, owner_id, count):
    # protection: проверяем входные значения
    if(count > 100): # todo: можно приделать scroll, то есть, разбить запрос на несколько, в каждом count не более 100
        raise AssertionError("count должен быть <= 100, но передали:" + str(count))

    # основная часть
    results = api.wall.get(owner_id=owner_id, filter = 'owner', count = count, offset=0)
    print("obtainWallItems, owner_id=" + str(owner_id) + "retrieve " + str(len(results['items'])) + ", of total " + str(results['count']))
    return results['items']

# для напоминания - сохраним в комментариях альтернативный метод vk_api для получения сообщений со стены сообщества
#tools = vk_api.VkTools(vk_session)
#wall = tools.get_all('wall.get', 10, {'owner_id': -92718200, 'filter':'owner'})

In [135]:
# записываем json в файл
def writeJsonToFile(fileName, jsonData):
    with io.open(fileName, 'w', encoding='utf-8') as f:
        f.write(json.dumps(jsonData, ensure_ascii=False, sort_keys=True, indent=4))

In [146]:
groups = obtainGroups(api, 'bitcoin', COUNT_GROUPS)
writeJsonToFile(fileNameFunc('groups'), groups)

obtainGroups, keyword=bitcoin, count=2


In [147]:
# отфильтруем сообщества - нам нужны только те, у которых есть доступ к сообщениям на стене
group_ids = []
for group in groups:
    if(group['is_closed'] == 0):
        group_ids.append(group['id'])
    else:
        print('skip group:' + str(group['id']))
print('total groups для обработки: ' + str(len(group_ids)))
# obtainWallItems(api=vk_session.get_api(), count=1, owner_id=-92718200)

total groups для обработки: 2


In [148]:
# для каждого сообщества - получаем сообщения со стены и записваем в соотв.файл
counter = 1 # счетчик, используем в названии файлов
for group_id in group_ids:
    vk_id = -1 * group_id # vk_api.wall.get по наличию знака минус - определяет, что требуется именно сообщество
    items = obtainWallItems(api=api, owner_id=vk_id, count=COUNT_MESSAGES_PER_GROUP)
    rawFileName = str(counter) + "_" + str(group_id)
    writeJsonToFile(fileNameFunc(rawFileName), items)
    counter = counter + 1

obtainWallItems, owner_id=-46371384retrieve 100, of total 48227
obtainWallItems, owner_id=-92718200retrieve 100, of total 10600
